In [1]:
import os
import glob
from osgeo import gdal
import numpy as np

# 设置GDAL不要压缩输出文件
gdal.SetConfigOption('COMPRESS_OVERVIEW', 'NONE')
gdal.UseExceptions()  # 启用GDAL异常

def process_file(input_file, output_file, target_srs='EPSG:4544', target_res=(0.1, 0.1)):
    try:
        # 如果输出文件已存在，跳过处理
        if os.path.exists(output_file):
            print(f"输出文件已存在，跳过处理: {output_file}")
            return output_file

        # 打开输入文件
        ds = gdal.Open(input_file)
        if ds is None:
            print(f"无法打开文件: {input_file}")
            return None

        # 设置转换选项
        options = gdal.WarpOptions(
            dstSRS=target_srs,
            xRes=target_res[0], yRes=target_res[1],
            resampleAlg=gdal.GRA_Bilinear,
            format='GTiff',
            creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
            warpOptions=['CUTLINE_ALL_TOUCHED=TRUE'],
            multithread=True,
            warpMemoryLimit=4096  # 限制内存使用为4GB
        )

        # 执行转换
        print(f"正在处理文件: {input_file}")
        gdal.Warp(output_file, ds, options=options)
        
        # 关闭数据集
        ds = None
        print(f"成功处理文件: {input_file}")
        return output_file

    except Exception as e:
        print(f"处理文件 {input_file} 时出错: {str(e)}")
        return None

def process_and_merge_images(input_folder, output_file):
    # 获取输入文件夹中所有的tif和ecw文件
    input_files = glob.glob(os.path.join(input_folder, '**', '*.tif'), recursive=True) + \
                  glob.glob(os.path.join(input_folder, '**', '*.ecw'), recursive=True)

    if not input_files:
        print("没有找到TIF或ECW文件")
        return

    # 创建一个列表来存储处理后的文件
    processed_files = []

    for input_file in input_files:
        output_file_temp = os.path.splitext(input_file)[0] + '_processed.tif'
        processed = process_file(input_file, output_file_temp)
        if processed:
            processed_files.append(processed)

    if not processed_files:
        print("没有成功处理的文件")
        return

    try:
        # 合并所有处理后的文件
        print("开始合并处理后的文件")
        options = gdal.WarpOptions(
            format='GTiff',
            dstNodata=None,
            creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
            multithread=True,
            warpMemoryLimit=4096  # 限制内存使用为4GB
        )
        
        gdal.Warp(output_file, processed_files, options=options)
        print(f"成功合并文件，输出: {output_file}")

    except Exception as e:
        print(f"合并文件时出错: {str(e)}")

    finally:
        # 清理临时文件
        for file in processed_files:
            try:
                os.remove(file)
                print(f"删除临时文件: {file}")
            except Exception as e:
                print(f"删除临时文件 {file} 时出错: {str(e)}")

    print(f"处理完成，输出文件: {output_file}")

In [ ]:
# 使用示例
input_folder = r'F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像'
output_file = r'F:\result\merged4.tif'
process_and_merge_images(input_folder, output_file)

In [ ]:
import arcpy

def convert_ecw_to_tif(input_file, output_file):
    arcpy.CopyRaster_management(input_file, output_file, pixel_type="8_BIT_UNSIGNED")

In [ ]:
ecw_path = r'C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.ecw'
output_path = r'C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.tif'

In [ ]:
convert_ecw_to_tif(ecw_path,output_path)

In [ ]:
import arcpy
import os

def convert_ecw_to_tif(input_file, output_file, target_crs='4544', cell_size=0.1):
    try:
        # 检查输入文件是否存在
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"输入文件不存在: {input_file}")
        
        # 检查输出路径是否可写
        output_dir = os.path.dirname(output_file)
        if not os.access(output_dir, os.W_OK):
            raise PermissionError(f"没有写入权限: {output_dir}")
        
        # 设置输出坐标系
        output_sr = arcpy.SpatialReference(int(target_crs))
        
        # 设置环境变量
        arcpy.env.outputCoordinateSystem = output_sr
        arcpy.env.cellSize = cell_size
        
        print(f"开始转换: {input_file} -> {output_file}")
        print(f"目标坐标系: {target_crs}, 目标像元大小: {cell_size}")
        
        # 使用 CopyRaster 转换 ECW 到 TIF
        arcpy.CopyRaster_management(
            in_raster=input_file,
            out_rasterdataset=output_file,
            pixel_type="8_BIT_UNSIGNED",
            nodata_value="255",
            format="TIFF",
            transform="Transform"
        )
        
        print(f"转换完成。输出文件: {output_file}")
        
    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(f"发生错误: {str(e)}")

# 使用示例
# input_ecw = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.ecw"
# output_tif = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1111.tif"
# convert_ecw_to_tif(input_ecw, output_tif, target_crs='4544', cell_size=0.1)

In [ ]:
input_ecw = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇2.ecw"
output_tif = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1111.tif"

In [ ]:
convert_ecw_to_tif(input_ecw, output_tif)